In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import folium
from folium.plugins import MarkerCluster
import googlemaps
import time
import math
from sklearn.model_selection import train_test_split
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.compose import make_column_transformer
from tqdm import tqdm
import mpld3

Important links
Simlar Competition
https://www.kaggle.com/code/headsortails/nyc-taxi-eda-update-the-fast-the-curious/report#fastest-routes
https://www.kaggle.com/code/karelrv/nyct-from-a-to-z-with-xgboost-tutorial

Where I got the taxi data 
https://catalog.data.gov/dataset/taxi-trips-in-2024

Where I got the weather data
https://www.visualcrossing.com/weather/weather-data-services

Quotes
https://kandmcollision.com/car-mean-time/#:~:text=What%20you%20don't%20know,moon%20and%20back%203%20times!

In [ ]:
taxi = pd.read_csv('curve.csv', low_memory=False)

In [ ]:
numerical_data = taxi.select_dtypes(include=['number'])
numerical_data = numerical_data.drop(['ORIGIN_BLOCK_LATITUDE', 
                                     'ORIGIN_BLOCK_LONGITUDE','DESTINATION_BLOCK_LAT', 'DESTINATION_BLOCK_LONG', 'Duration(m)',
      'duration', 'distance', 'precipprob', 'snow', 'snowdepth', 'MILEAGE', 'steps', 'Day_Monday',
       'Day_Saturday', 'Day_Sunday', 'Day_Thursday', 'Day_Tuesday', 'snowfall', 'precipitation',
       'Day_Wednesday'
       ], axis=1)
correlation_matrix = numerical_data.corr()

# Create an interactive heatmap with Plotly
fig = px.imshow(
    correlation_matrix,
    text_auto=".2f",
    title="Correlation Matrix Heatmap"
)
# Save as an HTML file
fig.write_html("matrix.html")

In [ ]:
def compute_center(X):
    return X.assign(
        CenterLat=(X["ORIGIN_BLOCK_LATITUDE"] + X["DESTINATION_BLOCK_LAT"]) / 2,
        CenterLong=(X["ORIGIN_BLOCK_LONGITUDE"] + X["DESTINATION_BLOCK_LONG"]) / 2
    )
def compute_direction(X):
    return X.assign(
        Direction=X.apply(
            lambda row: (
                'NorthEast' if row['DESTINATION_BLOCK_LAT'] > row['ORIGIN_BLOCK_LATITUDE'] and row['DESTINATION_BLOCK_LONG'] > row['ORIGIN_BLOCK_LONGITUDE'] else
                'NorthWest' if row['DESTINATION_BLOCK_LAT'] > row['ORIGIN_BLOCK_LATITUDE'] and row['DESTINATION_BLOCK_LONG'] < row['ORIGIN_BLOCK_LONGITUDE'] else
                'SouthEast' if row['DESTINATION_BLOCK_LAT'] <= row['ORIGIN_BLOCK_LATITUDE'] and row['DESTINATION_BLOCK_LONG'] >= row['ORIGIN_BLOCK_LONGITUDE'] else
                'SouthWest'
            ),
            axis=1
        )
    )
def haversine(lat1, lon1, lat2, lon2):
    R = 3959
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c
    
def compute_distance(X):
    return X.assign(
        Distance=(haversine(X["ORIGIN_BLOCK_LATITUDE"], X["ORIGIN_BLOCK_LONGITUDE"],X["DESTINATION_BLOCK_LAT"], X["DESTINATION_BLOCK_LONG"]))
    )

busy_roads = [
    'East Capitol Street Southeast',
    'New York Avenue Northeast',
    'Florida Avenue Northeast',
    '12'
]

def check_roads(x):
    for road in x:
        if str(road) in str(busy_roads):
            return 1
    return 0

In [ ]:
import pandas as pd
import ast

# List of busy roads
busy_roads = [
    'East Capitol Street Southeast',
    'New York Avenue Northeast',
    'Florida Avenue Northeast',
    'H St ',
    'Minnesota',
    'MLK',
    'Grant Street'
]

# Function to check if any road in the list is a busy road
def check_roads(road_list):
    for road in road_list:
        for busy_road in busy_roads:
            if busy_road in road:  # Substring match
                return 1
    return 0

# Apply the function to the 'roads' column
taxi['Congested_roads'] = taxi['roads'].apply(check_roads)

In [ ]:
plt.figure(figsize=(6, 5))
plt.hexbin(taxi['steps(OSRM)'], taxi['DURATION'], gridsize=30, cmap='Blues')
plt.colorbar(label='Density')
plt.title("Steps vs Duration(s)", fontsize=16)
plt.xlabel("Steps", fontsize=14)
plt.ylabel("Duration(s)", fontsize=14)
plt.xlim(0, 20)  # Adjusting x-axis range
plt.ylim(0,2500)
plt.grid(True)
plt.savefig('hexa.jpeg', format='jpeg')

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(
    x=taxi['steps(OSRM)'], 
    y=taxi['DURATION'], 
    cmap="Blues", fill=True, thresh=0, levels=100
)
plt.scatter(taxi['steps(OSRM)'], taxi['DURATION'], s=10, color="red", alpha=0.5)
plt.title("KDE Plot of steps(OSRM) vs DURATION", fontsize=16)
plt.xlabel("steps(OSRM)", fontsize=14)
plt.ylabel("DURATION", fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import Lasso
from sklearn.svm import SVR

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")
import logging

# Suppress specific LightGBM warnings
logging.getLogger("lightgbm").setLevel(logging.ERROR)
# Define Preprocessors
num_features = X_train.select_dtypes(include=["int64", "float64"]).columns
cat_features = X_train.select_dtypes(include=["object"]).columns

# Preprocessors
num_preprocessor = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

cat_preprocessor = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", max_categories=20))
])

preprocessor = ColumnTransformer([
    ("num", num_preprocessor, num_features),
    ("cat", cat_preprocessor, cat_features)
])

# Models
models = {
    'Ridge': Pipeline([("preprocessor", preprocessor), ("model", Ridge())]),
    #'Lasso': Pipeline([("preprocessor", preprocessor), ("model", Lasso(max_iter=50000))]),
  #  'ElasticNet': Pipeline([("preprocessor", preprocessor), ("model", ElasticNet(max_iter=50000))]),
  #  'SVR': Pipeline([("preprocessor", preprocessor), ("model", SVR())]),
   # 'RandomForest': Pipeline([("preprocessor", preprocessor), ("model", RandomForestRegressor(
 #      n_estimators=300, max_depth=15, min_samples_split=5, min_samples_leaf=2
  #  ))]),
 #   'GradientBoosting': Pipeline([("preprocessor", preprocessor), ("model", GradientBoostingRegressor())]),
 #   'XGBoost': Pipeline([("preprocessor", preprocessor), ("model", XGBRegressor(
#        n_estimators=1000, eval_metric="rmse", use_label_encoder=False
#    ))]),
 #   'LightGBM': Pipeline([("preprocessor", preprocessor), ("model", LGBMRegressor(
 #       feature_fraction=0.8,
 #       min_data_in_leaf=10,
 #       force_row_wise=True,
 #       verbose=-1
 #   ))]),
 #   'CatBoost': Pipeline([("preprocessor", preprocessor), ("model", CatBoostRegressor(
 #       verbose=0, early_stopping_rounds=10
 #   ))]),
}

# Parameter Grids
param_grids = {
    'Ridge': {'model__alpha': [50,100,200]},
   # 'Lasso': {'model__alpha': [0.001, 0.01,0.1]},
   # 'ElasticNet': {'model__alpha': [0.0001, 0.001], 'model__l1_ratio': [0.5, 0.9]},
   # 'SVR': {'model__C': [0.1, 1, 10], 'model__epsilon': [0.1]},
   # 'RandomForest': {
 #      'model__n_estimators': [100, 200],
   #     'model__max_depth': [10, 2],
   #     'model__min_samples_split': [2, 5, 10],
   #     'model__min_samples_leaf': [1,4],
     #   'model__max_features': ['sqrt', 'log2', None],
    #    'model__bootstrap': [True, False]
  #  }
#    'GradientBoosting': {'model__n_estimators': [100, 150, 200], 'model__learning_rate': [0.05, 0.1]},
#    'XGBoost': {
#        'model__n_estimators': [50, 100],
#        'model__learning_rate': [0.01, 0.1],
#        'model__max_depth': [3, 6],
#        'model__subsample': [0.8, 0.3],
#    },
#    'LightGBM': {
#        'model__n_estimators': [100, 200],
##        'model__learning_rate': [0.05, 0.1],
 #       'model__feature_fraction': [0.8, 1.0],
 #   },
 #   'CatBoost': {
 #   'model__iterations': [500],
 #   'model__depth': [6],
 #   'model__learning_rate': [0.01],
 #   'model__subsample': [0.8, 0.3],
 #   'model__bootstrap_type': ['Bayesian', 'Bernoulli']
 #   }
}

# GridSearchCV

for model_name, model in tqdm(models.items(), desc="Processing Models"):
    grid = GridSearchCV(model, param_grids[model_name], scoring=scorer, cv=5)
    grid.fit(X_train, y_train_log)
    results[model_name] = {
        'best_score': np.sqrt(-grid.best_score_),
        'best_params': grid.best_params_,
    }

resultssorted_results = sorted(results.items(), key=lambda x: x[1]['best_score'])
for model_name, result in sorted_results:
    print(f"{model_name}: RMSLE = {result['best_score']:.4f}, Best Params =